In [1]:
# Install dependencies
!pip install transformers torch pandas keybert scikit-learn tqdm rouge_score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=279ee51f06733298e1fdb91cdb580cb46388d98dac5622fd5d39ef5e3d213b89
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from keybert import KeyBERT
from torch.utils.data import Dataset
from tqdm import tqdm
import numpy as np
import os

# Preprocess dataset to extract title keywords
def preprocess_dataset(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    kw_model = KeyBERT()
    
    def extract_keywords(title):
        try:
            keywords = kw_model.extract_keywords(
                title, keyphrase_ngram_range=(1, 7), top_n=10
            )
            return ' '.join([kw[0] for kw in keywords])
        except:
            return ''
    
    df['title_keywords'] = df['title'].apply(extract_keywords)
    df.to_csv(output_csv, index=False)
    return df

# Custom Dataset for SciBERT Token Classification
class KeywordDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=512):
        self.df = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {'O': 0, 'B-KW': 1, 'I-KW': 2}  # BIO tagging scheme

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        abstract = str(self.df.iloc[idx]['abstract'])
        keywords = str(self.df.iloc[idx]['title_keywords']).split()

        # Tokenize abstract
        encoding = self.tokenizer(
            abstract,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Initialize labels for token classification (BIO scheme)
        labels = [self.label_map['O']] * self.max_length
        tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze())
        offset_mapping = encoding['offset_mapping'].squeeze()

        # Align keywords with tokens
        keyword_spans = []
        for keyword in keywords:
            keyword_lower = keyword.lower()
            start_idx = abstract.lower().find(keyword_lower)
            if start_idx != -1:
                end_idx = start_idx + len(keyword_lower)
                keyword_spans.append((start_idx, end_idx))

        # Assign BIO labels to tokens
        for i, (start, end) in enumerate(offset_mapping):
            if i >= len(tokens) or tokens[i] in ['[CLS]', '[SEP]', '[PAD]']:
                continue
            token_start, token_end = start, end
            for kw_start, kw_end in keyword_spans:
                if token_start >= kw_start and token_end <= kw_end:
                    if token_start == kw_start:
                        labels[i] = self.label_map['B-KW']  # Beginning of keyword
                    else:
                        labels[i] = self.label_map['I-KW']  # Inside keyword

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

# Training Function for SciBERT
def train_scibert(model, train_dataset, val_dataset, device, output_dir='/kaggle/working/scibert_finetuned'):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-5,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=200,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available(),
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer
    )

    trainer.train()

    # Save model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

# Main Execution
if __name__ == "__main__":
    # File paths
    train_csv = '/kaggle/input/springer-journal-final/train.csv'
    val_csv = '/kaggle/input/springer-journal-final/val.csv'
    processed_train_csv = '/kaggle/working/processed_train.csv'
    processed_val_csv = '/kaggle/working/processed_val.csv'

    # Preprocess datasets
    preprocess_dataset(train_csv, processed_train_csv)
    preprocess_dataset(val_csv, processed_val_csv)

    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = BertForTokenClassification.from_pretrained(
        'allenai/scibert_scivocab_uncased',
        num_labels=3  # O, B-KW, I-KW
    ).to(device)

    # Initialize datasets
    train_dataset = KeywordDataset(processed_train_csv, tokenizer)
    val_dataset = KeywordDataset(processed_val_csv, tokenizer)

    # Train SciBERT
    train_scibert(model, train_dataset, val_dataset, device)

    # Save keyword vocabulary (same as original)
    train_data = pd.read_csv(processed_train_csv)
    train_keywords = train_data['title_keywords'].apply(lambda x: x.split() if x else []).values
    keyword_counts = {}
    for sublist in train_keywords:
        for kw in sublist:
            keyword_counts[kw] = keyword_counts.get(kw, 0) + 1
    top_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:1000]
    keyword_vocab = [kw for kw, _ in top_keywords]
    with open('/kaggle/working/keyword_vocab.txt', 'w') as f:
        f.write('\n'.join(keyword_vocab))

2025-05-17 15:32:06.766170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747495926.992182      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747495927.057613      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.027800,0.028353
2,0.025500,0.027347
3,0.022700,0.028243


In [3]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, BertForTokenClassification, T5ForConditionalGeneration, T5Tokenizer
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
import os
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Preprocess dataset to extract title keywords
def preprocess_dataset(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    kw_model = KeyBERT()
    
    def extract_keywords(title):
        try:
            keywords = kw_model.extract_keywords(
                title, keyphrase_ngram_range=(1, 7), top_n=10
            )
            return ' '.join([kw[0] for kw in keywords])
        except:
            return ''
    
    df['title_keywords'] = df['title'].apply(extract_keywords)
    df.to_csv(output_csv, index=False)
    return df

# SciBERT-based keyword prediction function
def predict_keywords_scibert(abstract, scibert_model, scibert_tokenizer, max_len=512, device='cuda'):
    scibert_model.eval()
    encoding = scibert_tokenizer(
        abstract,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_offsets_mapping=True
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    offset_mapping = encoding['offset_mapping'].squeeze().cpu().numpy()
    
    with torch.no_grad():
        outputs = scibert_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()
    
    tokens = scibert_tokenizer.convert_ids_to_tokens(input_ids.squeeze())
    labels = ['O', 'B-KW', 'I-KW']
    predicted_labels = [labels[pred] for pred in predictions]
    
    # Reconstruct keywords from BIO tags
    keywords = []
    current_keyword = []
    for token, label, (start, end) in zip(tokens, predicted_labels, offset_mapping):
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
        if label == 'B-KW':
            if current_keyword:
                keywords.append(' '.join(current_keyword))
                current_keyword = []
            current_keyword.append(abstract[start:end])
        elif label == 'I-KW' and current_keyword:
            current_keyword.append(abstract[start:end])
        elif label == 'O' and current_keyword:
            keywords.append(' '.join(current_keyword))
            current_keyword = []
    
    if current_keyword:
        keywords.append(' '.join(current_keyword))
    
    # Clean and filter keywords
    keywords = [kw.strip() for kw in keywords if kw.strip()]
    return keywords

# Custom Dataset
class ScientificPaperDataset(Dataset):
    def __init__(self, csv_file, flan_tokenizer, scibert_model, scibert_tokenizer, max_length=512, device='cuda'):
        self.df = pd.read_csv(csv_file)
        self.flan_tokenizer = flan_tokenizer
        self.scibert_model = scibert_model
        self.scibert_tokenizer = scibert_tokenizer
        self.max_length = max_length
        self.device = device

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        abstract = str(self.df.iloc[idx]['abstract'])
        title = str(self.df.iloc[idx]['title'])
        title_keywords = str(self.df.iloc[idx]['title_keywords'])

        # Predict abstract keywords using SciBERT
        abstract_keywords = predict_keywords_scibert(
            abstract, self.scibert_model, self.scibert_tokenizer, device=self.device
        )
        abstract_keywords_text = ' '.join(abstract_keywords) if abstract_keywords else ''

        # Input for Flan-T5: SciBERT-predicted keywords + abstract
        input_text = f"Generate a concise and informative title based on this abstract of scientific research:\n{abstract}\nKeywords: {abstract_keywords_text}"

        input_encoding = self.flan_tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        title_encoding = self.flan_tokenizer(
            title,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'title_input_ids': title_encoding['input_ids'].squeeze(),
            'title_attention_mask': title_encoding['attention_mask'].squeeze(),
            'title_keywords': title_keywords,
            'abstract_keywords': abstract_keywords_text,
            'title': title
        }

# Multi-task Flan-T5 Model
class MultiTaskFlanT5(nn.Module):
    def __init__(self, flan_model_name='google/flan-t5-base'):
        super(MultiTaskFlanT5, self).__init__()
        self.flan = T5ForConditionalGeneration.from_pretrained(flan_model_name)
        self.tokenizer = T5Tokenizer.from_pretrained(flan_model_name)
        self.tfidf = TfidfVectorizer(max_features=1000)

    def forward(self, input_ids, attention_mask, title_input_ids, title_keywords, abstract_keywords):
        outputs = self.flan(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=title_input_ids
        )
        title_loss = outputs.loss

        try:
            keyword_vectors = self.tfidf.fit_transform([title_keywords, abstract_keywords]).toarray()
            title_kw_vector = torch.tensor(keyword_vectors[0], dtype=torch.float32)
            abstract_kw_vector = torch.tensor(keyword_vectors[1], dtype=torch.float32)
            cosine_sim = nn.functional.cosine_similarity(title_kw_vector, abstract_kw_vector, dim=0)
            keyword_loss = 1 - cosine_sim
        except:
            keyword_loss = torch.tensor(0.0, device=input_ids.device)

        return title_loss, keyword_loss

# Training and Validation Function
def train_and_validate(model, train_loader, val_loader, device, epochs=3, lr=5e-5, title_loss_weight=1.0, keyword_loss_weight=0.5):
    optimizer = AdamW(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            title_input_ids = batch['title_input_ids'].to(device)
            title_keywords = batch['title_keywords']
            abstract_keywords = batch['abstract_keywords']

            title_loss, keyword_loss = model(
                input_ids, attention_mask, title_input_ids,
                title_keywords, abstract_keywords
            )

            loss = title_loss_weight * title_loss + keyword_loss_weight * keyword_loss
            total_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                title_input_ids = batch['title_input_ids'].to(device)
                title_keywords = batch['title_keywords']
                abstract_keywords = batch['abstract_keywords']

                title_loss, keyword_loss = model(
                    input_ids, attention_mask, title_input_ids,
                    title_keywords, abstract_keywords
                )

                loss = title_loss_weight * title_loss + keyword_loss_weight * keyword_loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}, Average Validation Loss: {avg_val_loss:.4f}")

# Evaluation Function with ROUGE and BERTScore
def evaluate_model(model, test_loader, device):
    model.eval()
    generated_titles = []
    reference_titles = []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'rougeLsum': []}

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            titles = batch['title']

            # Generate titles
            generated_ids = model.flan.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=32,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True
            )
            generated_title = [model.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

            generated_titles.extend(generated_title)
            reference_titles.extend(titles)

            # Compute ROUGE scores
            for gen, ref in zip(generated_title, titles):
                scores = scorer.score(ref, gen)
                for metric in rouge_scores:
                    rouge_scores[metric].append(scores[metric].fmeasure)

    # Compute BERTScore
    try:
        P, R, F1 = bert_score(generated_titles, reference_titles, lang="en", verbose=True)
        bertscore_f1 = F1.mean().item()
    except:
        bertscore_f1 = 0.0

    # Average ROUGE scores
    avg_rouge = {metric: np.mean(scores) for metric, scores in rouge_scores.items()}

    # Print results
    print("\nEvaluation Results:")
    print(f"ROUGE-1: {avg_rouge['rouge1']:.4f}")
    print(f"ROUGE-2: {avg_rouge['rouge2']:.4f}")
    print(f"ROUGE-L: {avg_rouge['rougeL']:.4f}")
    print(f"ROUGE-Lsum: {avg_rouge['rougeLsum']:.4f}")
    print(f"BERTScore F1: {bertscore_f1:.4f}")

    # Save generated titles
    results_df = pd.DataFrame({
        'reference_title': reference_titles,
        'generated_title': generated_titles
    })
    results_df.to_csv('/kaggle/working/generated_titles.csv', index=False)

    return avg_rouge, bertscore_f1

# Main Execution
if __name__ == "__main__":
    # File paths
    train_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/train.csv'
    val_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/val.csv'
    test_csv = '/kaggle/input/springerjournal-450tk-0-7cosine/test.csv'
    processed_train_csv = '/kaggle/working/processed_train.csv'
    processed_val_csv = '/kaggle/working/processed_val.csv'
    processed_test_csv = '/kaggle/working/processed_test.csv'

    # Preprocess datasets
    preprocess_dataset(train_csv, processed_train_csv)
    preprocess_dataset(val_csv, processed_val_csv)
    preprocess_dataset(test_csv, processed_test_csv)

    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load SciBERT model and tokenizer for keyword prediction
    scibert_model_path = '/kaggle/working/scibert_finetuned'
    if not os.path.exists(scibert_model_path):
        raise FileNotFoundError(f"SciBERT model directory {scibert_model_path} does not exist. Please run the fine-tuning script first.")
    scibert_model = BertForTokenClassification.from_pretrained(scibert_model_path).to(device)
    scibert_tokenizer = AutoTokenizer.from_pretrained(scibert_model_path)

    # Initialize Flan-T5 tokenizer and datasets
    flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
    train_dataset = ScientificPaperDataset(
        processed_train_csv, flan_tokenizer, scibert_model, scibert_tokenizer, device=device
    )
    val_dataset = ScientificPaperDataset(
        processed_val_csv, flan_tokenizer, scibert_model, scibert_tokenizer, device=device
    )
    test_dataset = ScientificPaperDataset(
        processed_test_csv, flan_tokenizer, scibert_model, scibert_tokenizer, device=device
    )

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    # Initialize Flan-T5 model
    flan_model = MultiTaskFlanT5().to(device)

    # Train and validate
    train_and_validate(
        flan_model, train_loader, val_loader, device, epochs=3, lr=5e-5,
        title_loss_weight=1.0, keyword_loss_weight=0.5
    )

    # Evaluate on test set
    rouge_results, bertscore_result = evaluate_model(flan_model, test_loader, device)

    # Save the model
    flan_model.flan.save_pretrained('/kaggle/working/multi_task_flan_t5_model')
    flan_model.tokenizer.save_pretrained('/kaggle/working/multi_task_flan_t5_model')

    # Save keyword vocabulary
    train_data = pd.read_csv(processed_train_csv)
    train_keywords = train_data['title_keywords'].apply(lambda x: x.split() if x else []).values
    keyword_counts = {}
    for sublist in train_keywords:
        for kw in sublist:
            keyword_counts[kw] = keyword_counts.get(kw, 0) + 1
    top_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:1000]
    keyword_vocab = [kw for kw, _ in top_keywords]
    with open('/kaggle/working/keyword_vocab.txt', 'w') as f:
        f.write('\n'.join(keyword_vocab))

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.12 seconds, 198.74 sentences/sec

Evaluation Results:
ROUGE-1: 0.6616
ROUGE-2: 0.4737
ROUGE-L: 0.5659
ROUGE-Lsum: 0.5659
BERTScore F1: 0.9228
